# Merging and Joining

One important task that will come up again and again is merging data together. In Real Life&#8482; we rarely have a single dataset - we need to combine data from multiple sources,
Luckily Pandas makes it very simple to join together datasets! Let's start by reading in our previous data and we can think about some possible external data to read in

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
df = pd.read_csv('data/Consumo_cerveja.csv', 
                 decimal=',', 
                 thousands='.', 
                 header=0, 
                 names=['date','median_temp','min_temp','max_temp','precip','weekend','consumption'], 
                 parse_dates=['date'], 
                 nrows=365)

In [40]:
df.head()

,date,median_temp,min_temp,max_temp,precip,weekend,consumption
0,2015-01-01,27.30,23.9,32.5,0.0,0,25461
1,2015-01-02,27.02,24.5,33.5,0.0,0,28972
2,2015-01-03,24.82,22.4,29.9,0.0,1,30814
3,2015-01-04,23.98,21.5,28.6,1.2,1,29799
4,2015-01-05,23.82,21.0,28.3,0.0,0,28900


We have a timeseries - so holidays is always relevant to look at - is beer consumption affected by holidays? So let's find some holiday data! (and show off pandas `.read_html`!)
For all our informational needs, we turn to [wikipedia]('https://en.wikipedia.org/wiki/Public_holidays_in_Brazil'):

![warning](images/warning.resized.png) `.read_html` requires `lxml` to be installed

`.read_html` will read a webpage and intelligently find all `<table>` elements, converting those to dataframes - we always end up with a list of dataframes, even if there's only one!

In [41]:
holidays = pd.read_html('https://en.wikipedia.org/wiki/Public_holidays_in_Brazil', header=0)[0]
holidays

,Date,English name,Portuguese name,Remarks
0,January 1,New Year's Day,Ano Novo / Confraternização Universal,Celebrates the beginning of the year. Festivit...
1,April 21,Tiradentes,Dia de Tiradentes,Anniversary of considered the national martyr ...
2,May 1,Work Day,Dia do Trabalhador / Dia do Trabalho,Celebrates the achievements of workers and the...
3,September 7,Independence Day,Dia da Independência,Celebrates Brazil's declaration of independenc...
4,October 12,Our Lady of the Apparition,Nossa Senhora Aparecida,Commemorates the Virgin Mary as Nossa Senhora ...
5,November 2,All Souls' Day,Dia de Finados,Another Christian holiday; it commemorates the...
6,November 15,Republic Proclamation Day,Proclamação da República,Commemorates the end of the Empire of Brazil a...
7,December 25,Christmas,Natal,Celebrates the nativity of Jesus.


Now we have a beautiful dataframe straight from Wikipedia! We do have one more step before we are ready to merge - we need to turn `Date` into a `DateTime` type so we can match with our data. We have to remember to add a year, else pandas will interpret it as 1900.

In [42]:
holidays['Date'] = pd.to_datetime(holidays.Date + ', 2015', format='%B %d, %Y')
holidays

,Date,English name,Portuguese name,Remarks
0,2015-01-01,New Year's Day,Ano Novo / Confraternização Universal,Celebrates the beginning of the year. Festivit...
1,2015-04-21,Tiradentes,Dia de Tiradentes,Anniversary of considered the national martyr ...
2,2015-05-01,Work Day,Dia do Trabalhador / Dia do Trabalho,Celebrates the achievements of workers and the...
3,2015-09-07,Independence Day,Dia da Independência,Celebrates Brazil's declaration of independenc...
4,2015-10-12,Our Lady of the Apparition,Nossa Senhora Aparecida,Commemorates the Virgin Mary as Nossa Senhora ...
5,2015-11-02,All Souls' Day,Dia de Finados,Another Christian holiday; it commemorates the...
6,2015-11-15,Republic Proclamation Day,Proclamação da República,Commemorates the end of the Empire of Brazil a...
7,2015-12-25,Christmas,Natal,Celebrates the nativity of Jesus.


Now we have a nicely formatted dataframe, ready to join. We don't really want all the extra information, we are mostly interested in a binary holiday/not_holiday marker, so let's add a column of ones:

In [43]:
holidays['holiday'] = 1
holidays

,Date,English name,Portuguese name,Remarks,holiday
0,2015-01-01,New Year's Day,Ano Novo / Confraternização Universal,Celebrates the beginning of the year. Festivit...,1
1,2015-04-21,Tiradentes,Dia de Tiradentes,Anniversary of considered the national martyr ...,1
2,2015-05-01,Work Day,Dia do Trabalhador / Dia do Trabalho,Celebrates the achievements of workers and the...,1
3,2015-09-07,Independence Day,Dia da Independência,Celebrates Brazil's declaration of independenc...,1
4,2015-10-12,Our Lady of the Apparition,Nossa Senhora Aparecida,Commemorates the Virgin Mary as Nossa Senhora ...,1
5,2015-11-02,All Souls' Day,Dia de Finados,Another Christian holiday; it commemorates the...,1
6,2015-11-15,Republic Proclamation Day,Proclamação da República,Commemorates the end of the Empire of Brazil a...,1
7,2015-12-25,Christmas,Natal,Celebrates the nativity of Jesus.,1


To merge, we simply use `df.merge()` - it takes a number of options, so let's try merging this one first and we can work through them

In [44]:
df.merge(holidays, left_on='date', right_on='Date')

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,Date,English name,Portuguese name,Remarks,holiday
0,2015-01-01,27.30,23.9,32.5,0.0,0,25461,2015-01-01,New Year's Day,Ano Novo / Confraternização Universal,Celebrates the beginning of the year. Festivit...,1
1,2015-04-21,22.52,19.3,29.0,0.0,0,31108,2015-04-21,Tiradentes,Dia de Tiradentes,Anniversary of considered the national martyr ...,1
2,2015-05-01,17.34,13.4,22.0,0.0,0,17939,2015-05-01,Work Day,Dia do Trabalhador / Dia do Trabalho,Celebrates the achievements of workers and the...,1
3,2015-09-07,18.72,13.8,24.4,0.1,0,20227,2015-09-07,Independence Day,Dia da Independência,Celebrates Brazil's declaration of independenc...,1
4,2015-10-12,22.76,19.0,29.6,0.0,0,26249,2015-10-12,Our Lady of the Apparition,Nossa Senhora Aparecida,Commemorates the Virgin Mary as Nossa Senhora ...,1
5,2015-11-02,21.82,18.3,27.2,39.6,0,20648,2015-11-02,All Souls' Day,Dia de Finados,Another Christian holiday; it commemorates the...,1
6,2015-11-15,23.06,19.8,29.7,8.0,1,29386,2015-11-15,Republic Proclamation Day,Proclamação da República,Commemorates the end of the Empire of Brazil a...,1
7,2015-12-25,23.58,17.8,31.0,39.0,0,26308,2015-12-25,Christmas,Natal,Celebrates the nativity of Jesus.,1


First off, we are only getting 8 rows back - why is that? 

By default, `.merge` assumes you want to do an `inner join` - for those of you who know SQL, this makes perfect sense :-)

An inner join returns only those rows where we can match the key in both datasets - in this case we keep only those days that are in both our consumption dataset and our holidays dataset.

What we want is to keep all rows in our consumption dataset and add on the rows from holiday that match  - we want a `left join` (This is of course relative to which dataframe we call `.merge` on)

In [45]:
df_holidays = df.merge(holidays, left_on='date', right_on='Date', how='left')
df_holidays

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,Date,English name,Portuguese name,Remarks,holiday
0,2015-01-01,27.30,23.9,32.5,0.0,0,25461,2015-01-01,New Year's Day,Ano Novo / Confraternização Universal,Celebrates the beginning of the year. Festivit...,1.0
1,2015-01-02,27.02,24.5,33.5,0.0,0,28972,NaT,NaN,NaN,NaN,NaN
2,2015-01-03,24.82,22.4,29.9,0.0,1,30814,NaT,NaN,NaN,NaN,NaN
3,2015-01-04,23.98,21.5,28.6,1.2,1,29799,NaT,NaN,NaN,NaN,NaN
4,2015-01-05,23.82,21.0,28.3,0.0,0,28900,NaT,NaN,NaN,NaN,NaN
5,2015-01-06,23.78,20.1,30.5,12.2,0,28218,NaT,NaN,NaN,NaN,NaN
6,2015-01-07,24.00,19.5,33.7,0.0,0,29732,NaT,NaN,NaN,NaN,NaN
7,2015-01-08,24.90,19.5,32.8,48.6,0,28397,NaT,NaN,NaN,NaN,NaN
8,2015-01-09,28.20,21.9,34.0,4.4,0,24886,NaT,NaN,NaN,NaN,NaN
9,2015-01-10,26.76,22.1,34.2,0.0,1,37937,NaT,NaN,NaN,NaN,NaN


Now we have all our rows and pandas simply fills in `NaN` in those rows that don't match. In order to get our binary holiday marker, we simply `.fillna()` our holiday column

In [46]:
df_holidays['holiday'] = df_holidays.holiday.fillna(0)
df_holidays

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,Date,English name,Portuguese name,Remarks,holiday
0,2015-01-01,27.30,23.9,32.5,0.0,0,25461,2015-01-01,New Year's Day,Ano Novo / Confraternização Universal,Celebrates the beginning of the year. Festivit...,1.0
1,2015-01-02,27.02,24.5,33.5,0.0,0,28972,NaT,NaN,NaN,NaN,0.0
2,2015-01-03,24.82,22.4,29.9,0.0,1,30814,NaT,NaN,NaN,NaN,0.0
3,2015-01-04,23.98,21.5,28.6,1.2,1,29799,NaT,NaN,NaN,NaN,0.0
4,2015-01-05,23.82,21.0,28.3,0.0,0,28900,NaT,NaN,NaN,NaN,0.0
5,2015-01-06,23.78,20.1,30.5,12.2,0,28218,NaT,NaN,NaN,NaN,0.0
6,2015-01-07,24.00,19.5,33.7,0.0,0,29732,NaT,NaN,NaN,NaN,0.0
7,2015-01-08,24.90,19.5,32.8,48.6,0,28397,NaT,NaN,NaN,NaN,0.0
8,2015-01-09,28.20,21.9,34.0,4.4,0,24886,NaT,NaN,NaN,NaN,0.0
9,2015-01-10,26.76,22.1,34.2,0.0,1,37937,NaT,NaN,NaN,NaN,0.0


We got a lot of junk now that we are not interested in, particulary the duplication of the Date columns - I could simply drop them, but let's look at how we can remove them from the merge alltogether

In [47]:
merge_holidays = holidays.rename(columns={"Date": "date"})[["date", "holiday"]]
df_holidays = df.merge(merge_holidays, on='date', how='left').assign(holiday=lambda x: x.holiday.fillna(0))
df_holidays

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,holiday
0,2015-01-01,27.30,23.9,32.5,0.0,0,25461,1.0
1,2015-01-02,27.02,24.5,33.5,0.0,0,28972,0.0
2,2015-01-03,24.82,22.4,29.9,0.0,1,30814,0.0
3,2015-01-04,23.98,21.5,28.6,1.2,1,29799,0.0
4,2015-01-05,23.82,21.0,28.3,0.0,0,28900,0.0
5,2015-01-06,23.78,20.1,30.5,12.2,0,28218,0.0
6,2015-01-07,24.00,19.5,33.7,0.0,0,29732,0.0
7,2015-01-08,24.90,19.5,32.8,48.6,0,28397,0.0
8,2015-01-09,28.20,21.9,34.0,4.4,0,24886,0.0
9,2015-01-10,26.76,22.1,34.2,0.0,1,37937,0.0


If the keys have the same name in both datasets, we can use the `on` parameter, which will avoid duplication - we also select out only the columns we are interested in merging in the beginning. Now we have our prepared data!

# Joining

Joining is a special case of merging, where we simply merge on the index - this can be useful when we know our indexes are the same.

The main differences are that `.join` defaults to a `left join` and it only joins on indexes

In [48]:
df = df.set_index('date')
holidays = holidays.set_index('Date')

In [49]:
df.join(holidays['holiday']).assign(holiday=lambda x: x.holiday.fillna(0))

,median_temp,min_temp,max_temp,precip,weekend,consumption,holiday
date,,,,,,,
2015-01-01,27.30,23.9,32.5,0.0,0,25461,1.0
2015-01-02,27.02,24.5,33.5,0.0,0,28972,0.0
2015-01-03,24.82,22.4,29.9,0.0,1,30814,0.0
2015-01-04,23.98,21.5,28.6,1.2,1,29799,0.0
2015-01-05,23.82,21.0,28.3,0.0,0,28900,0.0
2015-01-06,23.78,20.1,30.5,12.2,0,28218,0.0
2015-01-07,24.00,19.5,33.7,0.0,0,29732,0.0
2015-01-08,24.90,19.5,32.8,48.6,0,28397,0.0
2015-01-09,28.20,21.9,34.0,4.4,0,24886,0.0


# Concatenation

Concatenation also comes up often, you can concatenate on both axes - adding more columns and adding more rows.

Let's say I want to look only at the top10 and bottom10 days per consumption

In [68]:
top10 = df_holidays.sort_values(by='consumption', ascending=False).reset_index(drop=True).head(10)
bottom10 = df_holidays.sort_values(by='consumption').reset_index(drop=True).head(10)

In [69]:
top10

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,holiday
0,2015-01-10,26.76,22.1,34.2,0.0,1,37937,0.0
1,2015-01-17,28.86,22.0,35.8,0.0,1,37690,0.0
2,2015-01-11,27.62,22.2,34.8,3.4,1,36254,0.0
3,2015-10-15,28.12,18.9,35.8,0.0,0,35861,0.0
4,2015-10-31,23.18,18.2,29.3,0.0,1,35781,0.0
5,2015-01-20,27.68,23.3,35.6,0.6,0,35127,0.0
6,2015-09-20,23.60,19.2,33.3,0.0,1,34695,0.0
7,2015-10-11,22.48,19.6,30.0,0.0,1,34496,0.0
8,2015-11-28,23.70,18.7,29.8,0.0,1,34382,0.0
9,2015-09-17,27.26,20.4,33.0,0.0,0,33930,0.0


In [70]:
bottom10

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,holiday
0,2015-07-22,14.52,12.8,16.1,0.0,0,14343,0.0
1,2015-07-03,17.52,15.6,18.4,17.8,0,14940,0.0
2,2015-06-01,16.28,15.2,18.0,2.2,0,16228,0.0
3,2015-06-19,14.66,13.2,17.7,3.8,0,16748,0.0
4,2015-05-11,17.86,15.7,20.6,22.2,0,16956,0.0
5,2015-09-09,18.82,15.8,22.9,58.0,0,16977,0.0
6,2015-09-08,17.08,15.8,19.2,23.6,0,17075,0.0
7,2015-06-24,14.54,13.9,18.0,3.8,0,17241,0.0
8,2015-05-13,16.34,14.7,18.9,2.8,0,17287,0.0
9,2015-02-17,22.76,19.7,28.0,85.3,0,17399,0.0


If I want to compare them easily, I can just concatenate them together

In [71]:
pd.concat([bottom10, top10]).sort_values(by='consumption', ascending=False)

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,holiday
0,2015-01-10,26.76,22.1,34.2,0.0,1,37937,0.0
1,2015-01-17,28.86,22.0,35.8,0.0,1,37690,0.0
2,2015-01-11,27.62,22.2,34.8,3.4,1,36254,0.0
3,2015-10-15,28.12,18.9,35.8,0.0,0,35861,0.0
4,2015-10-31,23.18,18.2,29.3,0.0,1,35781,0.0
5,2015-01-20,27.68,23.3,35.6,0.6,0,35127,0.0
6,2015-09-20,23.60,19.2,33.3,0.0,1,34695,0.0
7,2015-10-11,22.48,19.6,30.0,0.0,1,34496,0.0
8,2015-11-28,23.70,18.7,29.8,0.0,1,34382,0.0
9,2015-09-17,27.26,20.4,33.0,0.0,0,33930,0.0


What if I want to compare the consumption side by side?

In [74]:
pd.concat([bottom10.consumption, top10.consumption], axis=1, keys=['bottom', 'top'])

,bottom,top
0,14343,37937
1,14940,37690
2,16228,36254
3,16748,35861
4,16956,35781
5,16977,35127
6,17075,34695
7,17241,34496
8,17287,34382
9,17399,33930
